In [1]:
import re
from dataclasses import dataclass
from pathlib import Path

data_file = Path("../Data/day3.txt").read_text()


EXAMPLE = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
EXAMPLE2 = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"


@dataclass
class Result:
    multiplication: tuple[int, int]
    start_index: int
    is_enabled: bool = False

    def multiply(self):
        return self.multiplication[0] * self.multiplication[1]


def adjust_results(results: list[Result], enabled_matches: list[tuple[bool, int]]):
    adjusted_results: list[Result] = []
    is_enabled = True

    def adjust_results_from(
        is_enabled: bool,
        current_enabled_match_start_index: int,
        next_enabled_match_start_index: int | None,
    ):
        for result_index in range(len(adjusted_results), len(results)):
            result = results[result_index]
            if result.start_index < current_enabled_match_start_index:
                return

            if next_enabled_match_start_index:
                if result.start_index > next_enabled_match_start_index:
                    return

            result.is_enabled = is_enabled
            adjusted_results.append(result)

    def adjust_results_to(is_enabled: bool, to_index: int):
        for index in range(len(adjusted_results), len(results)):
            result = results[index]
            if result.start_index > to_index:
                return

            result.is_enabled = is_enabled
            adjusted_results.append(result)

    for enable_index in range(len(enabled_matches)):
        enabled_match = enabled_matches[enable_index]
        if enable_index == 0:
            adjust_results_to(is_enabled, enabled_match[1])

        is_enabled = enabled_match[0]
        next_enabled_match_index = (
            enabled_matches[enable_index + 1][1]
            if (enable_index + 1) < len(enabled_matches)
            else None
        )
        adjust_results_from(is_enabled, enabled_match[1], next_enabled_match_index)

    # To whatever end if any 🐸
    adjust_results_to(is_enabled, 1_000_000)

    return adjusted_results


def prepare(input: str):
    multiplication_pattern = r"mul\((\d+),(\d+)\)"
    results = list(
        map(
            lambda match: Result(
                (int(match.group(1)), int(match.group(2))), match.start()
            ),
            re.finditer(multiplication_pattern, input),
        )
    )
    enable_pattern = r"(do|don't)\(\)"
    enabled_matches = list(
        map(
            lambda match: (match.group(0) == "do()", match.start()),
            re.finditer(enable_pattern, input),
        )
    )

    return adjust_results(results, enabled_matches)


example_data = prepare(EXAMPLE)
example2_data = prepare(EXAMPLE2)
data = prepare(data_file)

assert len(example2_data) == 4
assert example2_data[0].is_enabled is True
assert example2_data[1].is_enabled is False
assert example2_data[2].is_enabled is False
assert example2_data[3].is_enabled is True

In [2]:
def part1(input: list[Result]):
    return sum(map(lambda result: result.multiply(), input))


assert part1(example_data) == 161

result = part1(data)

assert result == 166357705

print("result is", result)

result is 166357705


In [3]:
def part2(input: list[Result]):
    return sum(
        map(
            lambda result: result.multiply(),
            filter(lambda result: result.is_enabled, input),
        )
    )


assert part2(example2_data) == 48

result = part2(data)

assert result == 88811886

print("result is", result)

result is 88811886
